#### Lecture 1. Transformer

Attention is all you need(2017)
- Attention module이 이전까지 LSTM / GRU 기반의 seq to seq 모델에 사용되었었다면 여기서는 RNN이나 CNN module을 완전히 대체
- RNN 의 Long term dependency 를 해결하기 위한 시도 (Bi-directional RNN)
- sequence에 대해 각 단어별로 seqeunce의 문맥을 반영하여 encoding vector를 생성하는 RNN의 대체 모델로 부상

Transformer에서 attention 적용 방식
- input vector가 attention module에서 decoder의 hidden state vector 처럼 역할 수행
- 주어진 입력벡터와의 내적에 기반한 유사도를 구하여 softmax output 으로 작용
- 주어진 벡터들중에 어느 벡터를 선별적으로 가져올 것인가 : query vector
- query vector와 내적이 되는 재료 벡터들 : key vector
- 구해진 유사도를 바탕으로 가중평균이 구해지는 재료 벡터 : value vector
- i번째 벡터가 $W^Q$ 에 의해 query vector $q_i$ 를 생성하고 $W^K$, $W^V$ 에 의해 생성된 k1,...,kn, v1,...,vn 가 존재 하여 내적을 통해 encoding vector hi 생성
- 모든 k1,...,kn, v1,...,vn 을 고려하기 때문에 longterm dependency 근본적으로 해소

$$ A(q, K, V) = \sum_i \frac{exp(q \cdot k_i)}{\sum_j exp(q \cdot k_j)} v_i $$
$$ A(Q, K, V) = softmax(\frac{QK^T}{\sqrt{d_k}})V $$
- query key 의 차원 $d_k$, value의 차원 $d_v$, row-wise softmax
- 내적값의 분산이 query, key vector의 차원과 비례하므로 보간하기 위해 $\sqrt{d_k}$로 나눠줌
$$ (|Q|\times d_k)\times(d_k\times|K|)\times(|V|\times d_v) = (|Q|\times d_v) $$
- 병렬적인 행렬 연산을 통해 빠른 학습이 가능

#### Lecture 2. Transformer (continued)

Multihead Attention (MHA)
- 여러개의 $W^Q, W^K, W^V$ 가 존재, 서로 다른 버전의 encoding vector 생성하여 concat
- 동일한 sequence가 주어졌을 때에도 특정한 query에 대해 여러 정보를 추출할 필요가 있음
- concat된 vector에 선형 변환을 통해 차원을 축소하고 최종 결과를 얻어냄
- n : sequence length, d : dimension 일때 complexity per layer
    - self-attention : $O(n^2 \cdot d) $ GPU로 병렬 처리 가능 $O(1)$
    - recurrent : $O(n \cdot d^2) $ GPU로 병렬 처리 불가 $O(n)$
    - 시간은 recurrent가 메모리는 self-attention이 많이 요구됨
    |        | complexity per layer | sequential<br> operations | Maximum<br>Pathlength |
| --------- | ------------------ | --------------------- | ------------------ |
| self-attention | $O(n^2 \cdot d)$   | $O(1)$          |      $O(1)   $        |
| recurrent      | $O(n \cdot d^2)$   | $O(n)$          |     $ O(n)  $         |



Transformer 구조
- 각각의 block은 MHA와 two-layer feed-forward NN(with ReLU)로 구성됨
- 두단계 모두 residual connection 수행(Add & layer Norm) - gradient vanishing 해결
- Normalization - 평균을 0, 분산을 1로 변환시키는 과정
    - Group Norm, Batch Norm, Instance Norm, Layer Norm
- positional encoding
    - 단어의 순서를 무시한다는 특성을 보완하기 위한 기법
    - 위치정보를 가지고 있는 상수 벡터를 word 입력 벡터에 더해줌
- learning rate scheduling
    - 처음에는 minimum 과 거리가 먼 gradient가 가파른 위치에서 시작할 가능성 높음 >>> 초반에는 lr을 작게 시작하여 점점 증가시킴
    - global minimum으로 수렴할 수 있도록 동적으로 lr을 감소시킴
- 동일한 encoding block을 6회 통과시킨 후 decoder로 보냄
- interative 하게 특정 layer에서 query, key에 따른 attention pattern 시각화 가능
- decoder에서는 Masked self - MHA + encoder-decoder attention 수행
- linear transform 후 Output 출력
- masked self-attention : 학습 시 자기 자신 이후의 단어에 대한 attention 가중치를 0으로 설정